# Hyperparameter optimization with Maggy

*Note: this notebook needs to be run with a PySpark kernel to work properly!*

We'll use the [Maggy](https://maggy.ai/master/) library from Hopsworks to run experiments with hyperparameter tuning. 

The way it works is that we wrap training code in a function that we feed to an experiment object that executes the code. We can use code similar to the previous notebook, `4_model_training_and_registration.ipynb`.

The function will accept input parameters corresponding to model parameters that we want to tune. In this tutorial, a simple optimization we might want to do is to find a suitable value for the class weight on the positive class. With another model, for instance random forest or gradient boosting, you might want to tune the number of trees in the ensemble, the maximum depth of the trees, and so on.

Start by reading the training and validation data in the same way as for the previous notebook. We give the code without comments for brevity.

In [2]:
import hsfs
import pandas as pd

conn = hsfs.connection()
fs = conn.get_feature_store()
td = fs.get_training_dataset("transactions_dataset_splitted", version=1)
train_df = td.read('train')
val_df = td.read('validation')

if not type(train_df) == pd.core.frame.DataFrame: 
    train_df = train_df.toPandas()
    val_df = val_df.toPandas()
    
target = 'fraud_label'
features = list(set(train_df.columns) - set([target]))

X_train, y_train = train_df[features], train_df[target]
X_val, y_val = val_df[features], val_df[target]

Connected. Call `.close()` to terminate connection gracefully.

Now wrap the training code into a function that returns some metric we want to optimize. 

This code assumes that the `X_train`, `y_train` etc variables already exist in the namespace. You could also require them to be inputs to the `fraud_logreg_train` function but would then need to modify how you call the function in the last cell for example using Python's `partial` functionality.

In [3]:
def fraud_logreg_train(class_weight):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import precision_score
    clf = LogisticRegression(class_weight={0: 1, 1: class_weight}, solver='liblinear')
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    pos_prec = precision_score(y_true=y_val, y_pred=preds, pos_label=1)
    return pos_prec

Just check that it works.

In [4]:
fraud_logreg_train(500)

0.015901647125836617

Now we have to define the search space, meaning the interval to be searched for the best value. We are trying to optimize the positive-class precision, and since the positive class is very rare, the weight on it should probably be large. Let's try between 10 and 10000.

In [5]:
from maggy import Searchspace

sp = Searchspace(class_weight=('DOUBLE', [10, 10000]))

Hyperparameter added: class_weight

Now we can run the `lagom` method, which tries to find the best value. Lagom is a Swedish word that means "just right". We give it the training wrapper function and the search space we just defined. Since precision is a metric we want to be as high as possible, we use `direction=max`. 

`num_trials` is simply how many models will be trained; you should set this based on how much time you are willing to spend. We'll just do five trials here to showcase the functionality.

The `optimizer` is the strategy used to determine the next parameter value to try. We will just ise random search, which often works well in practice. You can read about alternatives [here](https://maggy.ai/master/hpo/strategies/).

In [6]:
from maggy import experiment

result = experiment.lagom(train_fn=fraud_logreg_train,
                            searchspace=sp,
                            optimizer='randomsearch',
                            direction='max',
                            num_trials=5,
                            name='fraud_lr'
                           )

Started Maggy Experiment: fraud_lr, application_1649849762999_0193, run 1

------ RandomSearch Results ------ direction(max) 
BEST combination {"class_weight": 2055.11695843451} -- metric 0.006215254998100894
WORST combination {"class_weight": 5460.944535964473} -- metric 0.0020668741922343866
AVERAGE metric -- 0.0030176014605893644
EARLY STOPPED Trials -- 0
Total job time 0 hours, 0 minutes, 46 seconds

Finished ExperimentWARN: Can't reach Maggy server. No progress information and logs available. Job continues running anyway.


In [7]:
result

{'best_id': '908fd86e654a748f', 'best_val': 0.006215254998100894, 'best_hp': {'class_weight': 2055.11695843451}, 'worst_id': '17dabed92656e58c', 'worst_val': 0.0020668741922343866, 'worst_hp': {'class_weight': 5460.944535964473}, 'avg': 0.0030176014605893644, 'metric_list': [0.0020785289476878263, 0.0026303675938712436, 0.0020668741922343866, 0.006215254998100894, 0.0020969815710524704], 'num_trials': 5, 'early_stopped': 0}